In [1]:
from gensim.models import KeyedVectors
import pandas as pd
from collections import Counter
import nltk
import re
import requests
import pickle
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import textwrap               # ➊ NEW

from tensorflow.python.distribute.strategy_combinations import parameter_server_strategy_1worker_2ps_1gpu

2025-07-29 13:53:54.383475: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-29 13:53:54.401132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753790034.418963 1664763 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753790034.422749 1664763 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753790034.433171 1664763 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,tokens_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,630,8839,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6527,82445,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,8905,170205,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,17961,193272,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3533,47878,[Meteorology/Earth sciences],exact_sciences


## Subcorpora word counts

In [78]:
disciplines_list = ['Medicine',
 'Astronomy/Astrology/Cosmography',
 'Biology',
 'Mathematics',
 'Meteorology/Earth sciences',
 'Physics',
 'Geography/Cartography',
 'Alchemy/Chemistry']

In [79]:
subcorpus_counts = []
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"

periods = [(1501, 1550),
                      (1551, 1600),
                      (1601, 1650),
                      (1651, 1700)]
periods_str = [str(per[0]) + "-" + str(per[1]) for per in periods]
for per_tup, per_str in zip(periods, periods_str):
    sents_n = 0
    tokens_n = 0
    ids = metadata_table_long[metadata_table_long["file_year"].between(per_tup[0], per_tup[1])]["id"]
    for id in ids:
        f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
        sents_n += len(f_sents_data)
        for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
            tokens_n += len(sent_data)
    subcorpus_counts.append({"subcorpus": per_str, "sents_n": sents_n, "tokens_n": tokens_n})
for discipline in disciplines_list:
    sents_n = 0
    tokens_n = 0
    ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
    for id in ids:
        f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
        sents_n += len(f_sents_data)
        for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
            tokens_n += len(sent_data)
    subcorpus_counts.append({"subcorpus": discipline, "sents_n": sents_n, "tokens_n": tokens_n})

In [80]:
source_path = "/srv/data/tome/tome-corpus/sents_data_id_jsons_v3-0/"
emlap_metadata = pd.read_csv("/srv/data/tome/tome-corpus/emlap_metadata.csv", sep=";")
ids = emlap_metadata["No."]

sents_n = 0
tokens_n = 0
for id in ids:
    f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
    sents_n += len(f_sents_data)
    for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
        tokens_n += len(sent_data)
subcorpus_counts.append({"subcorpus": "EMLAP", "sents_n": sents_n, "tokens_n": tokens_n})

In [82]:
subcorpus_counts_df = pd.DataFrame(subcorpus_counts)
subcorpus_counts_df

,subcorpus,sents_n,tokens_n
0,1501-1550,1018967,12779473
1,1551-1600,2679563,35876603
2,1601-1650,1423173,17932748
3,1651-1700,2019943,27211868
4,Medicine,3210357,36896246
5,Astronomy/Astrology/Cosmography,2095377,29907378
6,Biology,2032483,25073799
7,Mathematics,1415929,19814766
8,Meteorology/Earth sciences,1215475,14833414
9,Physics,1256840,16344953


In [6]:
subcorpus_counts_df.to_csv("../data/subcorpus_counts.csv", index=False)

## Subcorpora word counts plot

In [ ]:
colors = ["green"] * len(disciplines_list) + ["blue"] * len(periods_str) + ["red"]

In [ ]:
# 0. --- data shortcuts ---
df      = subcorpus_counts_df     # alias for brevity
heights = df['tokens_n']
colors  = colors                  # comes from your earlier code

# 1. --- wrap long labels (12 chars/line) ➋ NEW ---
wrapped_labels = [
    textwrap.fill(lbl, width=12, break_long_words=False, break_on_hyphens=False)
    for lbl in df['subcorpus']
]

# 2. --- bar plot ---
x = np.arange(len(df))           # ➌ integer positions
fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.bar(x, heights, color=colors)

# 3. --- value labels above bars (unchanged) ---
for bar, value in zip(bars, heights):
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height + heights.max() * 0.01,
        f'{value/1e6:.1f}M',
        ha='center', va='bottom', fontsize=10
    )

# 4. --- axes labels & title (unchanged) ---
ax.set_xlabel('Subcorpus')
ax.set_ylabel('Number of Tokens')
ax.set_title('Token Counts by Subcorpus')

# 5. --- set the new x-tick labels ➍ NEW ---
ax.set_xticks(x)
ax.set_xticklabels(
    wrapped_labels,
    rotation=60,            # steeper rotation
    ha='right',
    rotation_mode='anchor'
)

# 6. --- y-axis formatter (unchanged) ---
ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y, _: f'{y/1e6:.1f}M')
)

# 7. --- legend (unchanged) ---
legend_elements = [
    Patch(facecolor='green', label='Disciplines'),
    Patch(facecolor='blue',  label='Time Periods'),
    Patch(facecolor='red',   label='EMLAP')
]
ax.legend(handles=legend_elements, loc='upper right')

# 8. --- layout tweak ---
fig.subplots_adjust(bottom=0.20)   # a bit more space for wrapped labels
plt.tight_layout()


In [ ]:
fig.savefig("../figures/subcorpus_counts.png")

## LiLa embeddings

In [53]:
lasla = KeyedVectors.load_word2vec_format("/srv/data/lila/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec")
lasla.most_similar("seminarium")

[('carnarium', 0.8714821934700012),
 ('sudarium', 0.8584282398223877),
 ('uinarius', 0.8317396640777588),
 ('torcularium', 0.8269298672676086),
 ('armarium', 0.8191075325012207),
 ('milliarium', 0.8161847591400146),
 ('mortarium', 0.8139162063598633),
 ('asinarius', 0.7973706722259521),
 ('pomarium', 0.7952086329460144),
 ('olearius', 0.7895564436912537)]

In [54]:
lasla.most_similar("semen", topn=20)

[('sementis', 0.7769805788993835),
 ('germen', 0.65008145570755),
 ('legumen', 0.6492694020271301),
 ('materies', 0.62647545337677),
 ('sulcus', 0.6210276484489441),
 ('stercoro', 0.6130126118659973),
 ('genitalis', 0.6120802760124207),
 ('sumen', 0.6056757569313049),
 ('animans', 0.5975176095962524),
 ('elementa', 0.5954310297966003),
 ('seges', 0.5945314764976501),
 ('primordium', 0.5920283198356628),
 ('bipalium', 0.5892609357833862),
 ('omnigenus', 0.5877910256385803),
 ('ablaqueo', 0.5792651772499084),
 ('renascor', 0.573520839214325),
 ('enascor', 0.5693485736846924),
 ('abdomen', 0.5686165690422058),
 ('priuus', 0.5666219592094421),
 ('augmen', 0.5655485391616821)]

In [55]:
lasla.index_to_key[:10]

['</s>', 'sum', 'et', 'qui', 'in', 'non', 'hic', 'is', 'ut', 'ego']

In [56]:
operamaiora = KeyedVectors.load_word2vec_format("/srv/data/lila/opera-maiora-lemmas_skip_100.vec")
operamaiora.most_similar("anima")

[('intellectiuus', 0.7068129181861877),
 ('separo', 0.7005686163902283),
 ('sensifico', 0.6973943114280701),
 ('sensitiuus', 0.6771703362464905),
 ('corpus', 0.663750171661377),
 ('uegetabilis', 0.6309762597084045),
 ('unio', 0.6247175335884094),
 ('rationalis', 0.623648464679718),
 ('uegetatiuus', 0.618419885635376),
 ('corporeus', 0.60992830991745)]

In [57]:
operamaiora.most_similar("semen")

[('formatiuus', 0.7347741723060608),
 ('decidocaedo', 0.7292525768280029),
 ('sementis', 0.7260836362838745),
 ('decisio', 0.718953549861908),
 ('proauus', 0.708860456943512),
 ('menstruus', 0.7087300419807434),
 ('fetus', 0.7033671736717224),
 ('incubus', 0.6823228597640991),
 ('seminalis', 0.6802768111228943),
 ('spumosus', 0.6695918440818787)]

In [58]:
operamaiora_vocab = operamaiora.index_to_key
len(operamaiora_vocab)

6724

In [59]:
lasla_vocab = lasla.index_to_key
len(lasla_vocab)

11327

In [60]:
vocab_overlap = list(set(lasla_vocab) & set(operamaiora_vocab))
len(vocab_overlap)

3950

## Build vocabulary data

In [36]:
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"

In [37]:
id = os.listdir(source_path)[0]

In [38]:
f_sents_data = json.load(open(source_path + str(id), "rb"))
f_sents_data[:3]

[['604894',
  0,
  'Noua Acta Heluetica Physico-mathematico- Anatomico-botanico- Medica Tabulis Aeneis Illustrata Et In Usus Publicos Exarata.',
  [['Noua', 'nouus', 'ADJ', [0, 4]],
   ['Acta', '', 'VERB', [5, 9]],
   ['Heluetica', 'helueticus', 'ADJ', [10, 19]],
   ['Physico', 'physicus', 'PROPN', [20, 27]],
   ['-', '-', 'PUNCT', [27, 28]],
   ['mathematico-', '', 'ADJ', [28, 40]],
   ['Anatomico', 'anatomicus', 'PROPN', [41, 50]],
   ['-', '-', 'PUNCT', [50, 51]],
   ['botanico-', '', 'PRON', [51, 60]],
   ['Medica', 'medica', 'PROPN', [61, 67]],
   ['Tabulis', 'tabula', 'NOUN', [68, 75]],
   ['Aeneis', 'aeneus', 'ADJ', [76, 82]],
   ['Illustrata', 'illustro', 'VERB', [83, 93]],
   ['Et', 'et', 'CCONJ', [94, 96]],
   ['In', 'in', 'ADP', [97, 99]],
   ['Usus', 'usus', 'NOUN', [100, 104]],
   ['Publicos', 'publicus', 'PROPN', [105, 113]],
   ['Exarata', 'exaro', 'VERB', [114, 121]],
   ['.', '.', 'PUNCT', [121, 122]]]],
 ['604894',
  1,
  'Uolumen I. Basileae, Typis & Sumptibus Joanni

In [39]:
def get_filtered_counts_by_ids(ids, lowercase=True):
    vocab_counter = Counter() # initiate an empty counter of words - we will feed it iterarively on the go by all words we meet
    for id in ids: # for each work ID from our subset of IDs
        # based on the ID, open the file containing the lemmatized sentences
        try:
            f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
            sents_n = len(f_sents_data)
            tokens_n = 0
            lemmata = []
            for sent_data in f_sents_data:
                tokens_n += len(sent_data[3])
                sent_lemmata = [t[1] for t in sent_data[3] if t[2] in ["NOUN", "VERB", "ADJ", "PROPN"]]
                sent_lemmata = [re.sub(r"\W*|\d*", "", t) for t in sent_lemmata]
                sent_lemmata = [l for l in sent_lemmata if len(l) > 1]
                lemmata.extend(sent_lemmata)
            if lowercase:
                lemmata = [l.lower() for l in lemmata]
            #overview_dict[filename.partition(".")[0]] = {"sents_n" : sents_n, "tokens_n" : tokens_n}
            word_counts = dict(nltk.FreqDist(lemmata).most_common())
            vocab_counter.update(word_counts)
        except:
            pass
    return vocab_counter

In [40]:
ids = metadata_table_long[metadata_table_long["file_year"].between(1600, 1610)]["id"]
vocab = get_filtered_counts_by_ids(ids)

In [41]:
list(vocab.items())[:100]

[('amor', 436),
 ('animus', 907),
 ('do', 2857),
 ('possum', 5880),
 ('habeo', 6613),
 ('google', 697),
 ('uideo', 4251),
 ('ualeo', 441),
 ('facio', 5891),
 ('uita', 1278),
 ('dico', 6654),
 ('deus', 1036),
 ('fero', 1987),
 ('lux', 1082),
 ('oculus', 2649),
 ('nomen', 2136),
 ('dies', 5150),
 ('mens', 601),
 ('os', 6732),
 ('musa', 193),
 ('acidalius', 102),
 ('bonus', 1010),
 ('uis', 2453),
 ('fides', 612),
 ('janus', 98),
 ('uolo', 1823),
 ('nouus', 952),
 ('uiuo', 677),
 ('dolor', 1055),
 ('pectus', 599),
 ('manus', 1347),
 ('ingenium', 469),
 ('tempus', 4281),
 ('homo', 2138),
 ('amicus', 413),
 ('magnus', 3618),
 ('carmen', 185),
 ('cura', 397),
 ('orbis', 1154),
 ('ago', 1172),
 ('felix', 448),
 ('flos', 527),
 ('uenio', 1317),
 ('amo', 274),
 ('diuus', 159),
 ('cor', 2420),
 ('mors', 719),
 ('caput', 5098),
 ('eo', 368),
 ('numen', 111),
 ('annus', 5028),
 ('spes', 229),
 ('capio', 587),
 ('dignus', 426),
 ('credo', 611),
 ('puer', 442),
 ('res', 3639),
 ('ars', 1282),
 ('hono

In [14]:
# generate vocabularies for each half-century under scrutiny
periods = [(1501, 1550),
           (1551, 1600),
           (1601, 1650),
           (1651, 1700)]

periods_vocabs = []
full_vocabs = []
for per in periods:
    ids = metadata_table_long[metadata_table_long["file_year"].between(per[0], per[1])]["id"] # use the metadata table to extract IDs of all works falling into this period
    vocab = get_filtered_counts_by_ids(ids)
    periods_vocabs.append(vocab)

disciplines_list = ['Medicine',
 'Astronomy/Astrology/Cosmography',
 'Biology',
 'Mathematics',
 'Meteorology/Earth sciences',
 'Physics',
 'Geography/Cartography',
 'Alchemy/Chemistry']

disciplines_vocabs = []
for discipline in disciplines_list:
    ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
    vocab = get_filtered_counts_by_ids(ids)
    disciplines_vocabs.append(vocab)

In [42]:
source_path = "/srv/data/tome/tome-corpus/sents_data_id_jsons_v3-0/"
emlap_metadata = pd.read_csv("/srv/data/tome/tome-corpus/emlap_metadata.csv", sep=";")
ids = emlap_metadata["No."]
emlap_vocab = [get_filtered_counts_by_ids(ids)]


In [43]:
len(emlap_vocab[0])

62254

In [44]:
emlap_vocab[0].keys()

dict_keys(['possum', 'uideo', 'res', 'ars', 'aurum', 'fero', 'facio', 'uis', 'natura', 'opus', 'tempus', 'ignis', 'primus', 'longus', 'mens', 'dico', 'do', 'pars', 'labor', 'terra', 'uarius', 'summus', 'duco', 'magnus', 'metallum', 'quaero', 'uerus', 'uita', 'seruo', 'proprius', 'calor', 'caelum', 'mortalis', 'manus', 'pondus', 'mirus', 'corpus', 'argentum', 'color', 'deus', 'doceo', 'ago', 'refero', 'unda', 'uolo', 'tellus', 'puluis', 'semen', 'alo', 'quondam', 'paruus', 'uirtus', 'artifex', 'species', 'mons', 'nomen', 'purus', 'humanus', 'orbis', 'imus', 'tenuis', 'munus', 'grauis', 'animus', 'certus', 'homo', 'annus', 'peto', 'carmen', 'cura', 'pater', 'praesto', 'liquor', 'diuinus', 'uenio', 'ostendo', 'rego', 'profero', 'uas', 'os', 'pectus', 'relinquo', 'teneo', 'tendo', 'sacer', 'immensus', 'cano', 'modus', 'puto', 'sentio', 'credo', 'locus', 'queo', 'rex', 'pergo', 'uates', 'soleo', 'aequus', 'cerno', 'causa', 'usus', 'leuis', 'altus', 'pius', 'compono', 'sanctus', 'pateo', 'co

In [113]:
[t for t in emlap_vocab[0].most_common(5000) if "alch" in t[0]]

[('alchimia', 396),
 ('alchemia', 220),
 ('alchymia', 120),
 ('alchimista', 116),
 ('alchimicus', 60),
 ('alchymista', 56),
 ('alchymicus', 37),
 ('alchimias', 34),
 ('alchimius', 34)]

In [114]:
periods_vocabs[0].most_common(10)

[('dico', 43255),
 ('liber', 39559),
 ('pars', 33225),
 ('facio', 31814),
 ('habeo', 31690),
 ('res', 29069),
 ('possum', 25029),
 ('locus', 23102),
 ('uideo', 21536),
 ('primus', 19306)]

In [115]:
# only 5,000 most common words for each period...
topn = 5000
vocabs_topn_lists =[[t[0] for t in vocab.most_common(topn)] for vocab in periods_vocabs + disciplines_vocabs + emlap_vocab]

In [116]:
# build one list of words, including all from all these sets, i.e. their union
vocab_union = set.union(*[set(vocab) for vocab in vocabs_topn_lists])
len(vocab_union)

10056

In [117]:
# add there also the words appearing both in lasla and operamaiora
vocab_union = set.union(*[set(vocab) for vocab in vocabs_topn_lists] + [set(vocab_overlap)])
len(vocab_union)

11045

In [119]:
[t for t in vocab_union if "alch" in t]

['alchemia',
 'alchymia',
 'alchymista',
 'alchimista',
 'alchimias',
 'alchimicus',
 'alchimia',
 'alchimius',
 'alchymicus']

In [120]:
vocab_union_df = pd.DataFrame(vocab_union, columns = ["word"])
vocab_union_df

,word
0,uen
1,succi
2,phasis
3,helueticus
4,porcus
...,...
11040,jdem
11041,immutatio
11042,sculum
11043,citrino


In [121]:
# add the counts of these words from the vocabularies
def find_in_vocab(word, vocab):
    try:
        return vocab[word]
    except:
        return 0
for vocab, per in zip(periods_vocabs + disciplines_vocabs + emlap_vocab, periods_str + disciplines_list + ["EMLAP"]):
    vocab_union_df[per] = vocab_union_df["word"].apply(lambda x: find_in_vocab(x, vocab))

In [122]:
vocab_union_df.head(5)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP
0,uen,135,491,315,164,769,435,398,303,328,286,338,260,0
1,succi,116,586,116,152,668,35,359,78,118,95,37,56,173
2,phasis,69,234,616,678,140,1070,239,140,81,245,208,5,0
3,helueticus,20,68,37,141,94,39,124,17,61,25,177,22,4
4,porcus,158,1074,219,485,600,115,1375,55,234,135,263,84,66


In [123]:
vocab_union_df[vocab_union_df["word"].str.startswith("alch")]

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP
4213,alchemia,7,354,281,72,85,15,15,15,16,15,15,643,220
4534,alchymia,16,70,86,272,200,51,123,34,161,59,139,84,120
4661,alchymista,0,35,33,162,33,15,127,10,149,29,126,41,56
5414,alchimista,4,13,5,49,7,10,16,0,41,4,16,13,116
5627,alchimias,0,5,1,61,3,0,3,0,9,0,3,55,34
6787,alchimicus,0,9,3,20,5,3,2,0,2,0,2,22,60
7158,alchimia,11,31,49,187,66,33,36,20,51,20,38,185,396
8451,alchimius,0,4,1,2,0,2,0,0,1,0,0,3,34
10538,alchymicus,4,19,5,34,10,6,34,4,40,11,30,15,37


In [124]:
# calculate the mean value for sorting
vocab_union_df["mean"] = vocab_union_df[vocab_union_df.columns[1:]].mean(axis=1)
vocab_union_df.sort_values("mean", ascending=False, inplace=True)
vocab_union_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,47600,39787,12754,14098,58211.692308
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,50798,46481,29327,4593,50532.692308
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,58081,27901,14528,14367,48999.384615
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,40475,27720,11720,8377,46153.846154
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,38497,32771,10723,9810,46132.000000
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,40653,23879,11693,16068,45480.153846
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,44153,29452,13842,8935,40963.769231
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,36674,20706,9607,6047,34549.384615
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,27829,31499,7668,3810,33177.153846
10915,corpus,16979,63171,22871,31938,71457,42909,27375,19867,24931,32049,11300,8640,12498,29691.153846


In [125]:
vocab_union_df.tail(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
1600,sicubi,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7146,tantummodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
631,dummodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4224,o,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
9895,confestim,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1943,quousque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
5950,utique,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
9680,antea,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
10646,quocirca,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
96,usquequaque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [126]:
# Filter only for words appearing at least 5 times in each of the subcorpora
#filtered_vocab_df = vocab_union_df[
#    (vocab_union_df['1501-1550'] >= 5) &
#    (vocab_union_df['1551-1600'] >= 5) &
#    (vocab_union_df['1601-1650'] >= 5) &
#    (vocab_union_df['1651-1700'] >= 5)
#    ]

In [127]:
filtered_vocab_df = vocab_union_df.copy()

In [128]:
filtered_vocab_df.head(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,47600,39787,12754,14098,58211.692308
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,50798,46481,29327,4593,50532.692308
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,58081,27901,14528,14367,48999.384615
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,40475,27720,11720,8377,46153.846154
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,38497,32771,10723,9810,46132.000000
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,40653,23879,11693,16068,45480.153846
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,44153,29452,13842,8935,40963.769231
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,36674,20706,9607,6047,34549.384615
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,27829,31499,7668,3810,33177.153846
10915,corpus,16979,63171,22871,31938,71457,42909,27375,19867,24931,32049,11300,8640,12498,29691.153846


In [129]:
filtered_vocab_df.tail(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
1600,sicubi,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7146,tantummodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
631,dummodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4224,o,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
9895,confestim,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1943,quousque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
5950,utique,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
9680,antea,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
10646,quocirca,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
96,usquequaque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [130]:
filtered_vocab_df["in_lila_embeddings"] = filtered_vocab_df["word"].apply(lambda x: x in vocab_overlap)

In [131]:
filtered_vocab_df["in_lasla"] = filtered_vocab_df["word"].apply(lambda x: x in lasla_vocab)
filtered_vocab_df["in_operamaiora"] = filtered_vocab_df["word"].apply(lambda x: x in operamaiora_vocab)

In [132]:
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean,in_lila_embeddings,in_lasla,in_operamaiora
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,47600,39787,12754,14098,58211.692308,True,True,True
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,50798,46481,29327,4593,50532.692308,True,True,True
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,58081,27901,14528,14367,48999.384615,True,True,True
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,40475,27720,11720,8377,46153.846154,True,True,True
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,38497,32771,10723,9810,46132.000000,True,True,True
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,40653,23879,11693,16068,45480.153846,True,True,True
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,44153,29452,13842,8935,40963.769231,True,True,True
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,36674,20706,9607,6047,34549.384615,True,True,True
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,27829,31499,7668,3810,33177.153846,True,True,True
10915,corpus,16979,63171,22871,31938,71457,42909,27375,19867,24931,32049,11300,8640,12498,29691.153846,True,True,True


In [133]:
filtered_vocab_df["in_lasla"].sum()

5777

In [134]:
filtered_vocab_df["in_operamaiora"].sum()

4841

In [135]:
filtered_vocab_df["in_lila_embeddings"].sum()


3950

## Add relative frequencies

In [136]:
for col, total in zip(filtered_vocab_df.columns[1:14], subcorpus_counts_df["tokens_n"]):
    filtered_vocab_df[col + "_freq"] = filtered_vocab_df[col].apply(lambda x: x/total)

In [137]:
filtered_vocab_df.head(5)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,1651-1700_freq,Medicine_freq,Astronomy/Astrology/Cosmography_freq,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.002654,0.002901,0.002718,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.001951,0.002651,0.002199,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002824,0.002204,0.002505,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.001925,0.002586,0.002206,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002196,0.002213,0.002111,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807


In [138]:
filtered_vocab_df.columns[18:]

Index(['1501-1550_freq', '1551-1600_freq', '1601-1650_freq', '1651-1700_freq',
       'Medicine_freq', 'Astronomy/Astrology/Cosmography_freq', 'Biology_freq',
       'Mathematics_freq', 'Meteorology/Earth sciences_freq', 'Physics_freq',
       'Geography/Cartography_freq', 'Alchemy/Chemistry_freq', 'EMLAP_freq'],
      dtype='object')

In [139]:
filtered_vocab_df["mean_freq"] = filtered_vocab_df[filtered_vocab_df.columns[18:]
].mean(axis=1)
filtered_vocab_df.sort_values("mean_freq", ascending=False, inplace=True)
filtered_vocab_df.head(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Medicine_freq,Astronomy/Astrology/Cosmography_freq,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.002901,0.002718,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002651,0.002199,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002204,0.002505,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.002616,0.002105,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002213,0.002111,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.002586,0.002206,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397,0.002279
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,...,0.002016,0.002096,0.001686,0.001846,0.002739,0.002701,0.002085,0.002239,0.002556,0.002129
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,...,0.001662,0.001787,0.001883,0.001437,0.001897,0.002244,0.001466,0.001554,0.001730,0.001717
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,...,0.001377,0.001913,0.001626,0.001578,0.001717,0.001703,0.002230,0.001240,0.001090,0.001632
56,aqua,11747,41231,20765,34038,42778,18714,30884,14948,28253,...,0.001159,0.000626,0.001232,0.000754,0.001905,0.001437,0.001160,0.001966,0.005758,0.001575


In [140]:
filtered_vocab_df.head(10).round(6).to_csv("../data/filtered_vocab_df_top10.csv")

In [141]:
filtered_vocab_df.columns

Index(['word', '1501-1550', '1551-1600', '1601-1650', '1651-1700', 'Medicine',
       'Astronomy/Astrology/Cosmography', 'Biology', 'Mathematics',
       'Meteorology/Earth sciences', 'Physics', 'Geography/Cartography',
       'Alchemy/Chemistry', 'EMLAP', 'mean', 'in_lila_embeddings', 'in_lasla',
       'in_operamaiora', '1501-1550_freq', '1551-1600_freq', '1601-1650_freq',
       '1651-1700_freq', 'Medicine_freq',
       'Astronomy/Astrology/Cosmography_freq', 'Biology_freq',
       'Mathematics_freq', 'Meteorology/Earth sciences_freq', 'Physics_freq',
       'Geography/Cartography_freq', 'Alchemy/Chemistry_freq', 'EMLAP_freq',
       'mean_freq'],
      dtype='object')

In [ ]:
filtered_vocab_df

In [142]:
filtered_vocab_df["shared"] = filtered_vocab_df[['1501-1550', '1551-1600', '1601-1650', '1651-1700', 'Medicine', 'Astronomy/Astrology/Cosmography', 'Biology', 'Mathematics', 'Meteorology/Earth sciences', 'Physics', 'Geography/Cartography', 'Alchemy/Chemistry', 'EMLAP']].ge(10).all(axis=1)

In [184]:
(filtered_vocab_df["shared"] & filtered_vocab_df["in_lila_embeddings"]).sum()

3055

In [185]:
filtered_vocab_df["shared"] = filtered_vocab_df["shared"] & filtered_vocab_df["in_lila_embeddings"]

In [186]:
filtered_vocab_df["shared"].sum()

3055

## Remove remaining wrong words and save

In [187]:
!cp ../data/filtered_vocab_df.json /srv/data/tome/noscemus/filtered_vocab_df.json

In [188]:
filtered_vocab_df = filtered_vocab_df[filtered_vocab_df["word"].apply(lambda x: len(x) > 2)]

## Explorations for article

In [189]:
len(filtered_vocab_df)

10893

In [191]:
subcorpus_counts_df["vectors_tokens"] = subcorpus_counts_df.apply(lambda row: filtered_vocab_df[filtered_vocab_df[row["subcorpus"]]>= 10][row["subcorpus"]].sum(), axis=1)
subcorpus_counts_df["vectors_coverage"] = subcorpus_counts_df["vectors_tokens"] / subcorpus_counts_df["tokens_n"]
subcorpus_counts_df

,subcorpus,sents_n,tokens_n,vectors_tokens,vectors_coverage
0,1501-1550,1018967,12779473,4482114,0.350728
1,1551-1600,2679563,35876603,11875601,0.331012
2,1601-1650,1423173,17932748,6070591,0.338520
3,1651-1700,2019943,27211868,8871741,0.326025
4,Medicine,3210357,36896246,12552049,0.340199
5,Astronomy/Astrology/Cosmography,2095377,29907378,9673126,0.323436
6,Biology,2032483,25073799,8766982,0.349647
7,Mathematics,1415929,19814766,6275265,0.316696
8,Meteorology/Earth sciences,1215475,14833414,5155292,0.347546
9,Physics,1256840,16344953,5588394,0.341903


In [190]:
subcorpus_counts_df

,subcorpus,sents_n,tokens_n,vectors_tokens,vectors_coverage
0,1501-1550,1018967,12779473,4482114,0.350728
1,1551-1600,2679563,35876603,11875601,0.331012
2,1601-1650,1423173,17932748,6070591,0.338520
3,1651-1700,2019943,27211868,8871741,0.326025
4,Medicine,3210357,36896246,12552049,0.340199
5,Astronomy/Astrology/Cosmography,2095377,29907378,9673126,0.323436
6,Biology,2032483,25073799,8766982,0.349647
7,Mathematics,1415929,19814766,6275265,0.316696
8,Meteorology/Earth sciences,1215475,14833414,5155292,0.347546
9,Physics,1256840,16344953,5588394,0.341903


## Add simple english translation

In [176]:
filtered_vocab_df.head(5)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921,True,"say, call, tell"
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680,True,"book, volume, inner bark of a tree, book, book"
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563,True,"be able, can"
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367,True,"do, make, handle"
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310,True,"have, hold, possess, consider, think"


In [177]:
len(filtered_vocab_df)

10893

In [151]:
# simple latin API:
# https://www.latin-is-simple.com/api/

In [153]:
filtered_vocab_df[filtered_vocab_df["word"].str.startswith("alch")]

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Astronomy/Astrology/Cosmography_freq,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared
4213,alchemia,7,354,281,72,85,15,15,15,16,...,5.015485e-07,5.982340e-07,7.570112e-07,1.078646e-06,9.177145e-07,1.061769e-06,1.039912e-04,0.000063,1.560643e-05,False
7158,alchimia,11,31,49,187,66,33,36,20,51,...,1.103407e-06,1.435762e-06,1.009348e-06,3.438184e-06,1.223619e-06,2.689814e-06,2.991970e-05,0.000113,1.286429e-05,True
4534,alchymia,16,70,86,272,200,51,123,34,161,...,1.705265e-06,4.905519e-06,1.715892e-06,1.085387e-05,3.609677e-06,9.839055e-06,1.358516e-05,0.000034,7.997093e-06,True
4661,alchymista,0,35,33,162,33,15,127,10,149,...,5.015485e-07,5.065048e-06,5.046741e-07,1.004489e-05,1.774248e-06,8.918856e-06,6.630853e-06,0.000016,4.548115e-06,False
5414,alchimista,4,13,5,49,7,10,16,0,41,...,3.343657e-07,6.381163e-07,0.000000e+00,2.764030e-06,2.447239e-07,1.132553e-06,2.102465e-06,0.000033,3.334546e-06,False
10538,alchymicus,4,19,5,34,10,6,34,4,40,...,2.006194e-07,1.355997e-06,2.018697e-07,2.696615e-06,6.729906e-07,2.123537e-06,2.425922e-06,0.000011,1.761951e-06,False
6787,alchimicus,0,9,3,20,5,3,2,0,2,...,1.003097e-07,7.976454e-08,0.000000e+00,1.348307e-07,0.000000e+00,1.415691e-07,3.558018e-06,0.000017,1.728421e-06,False
5627,alchimias,0,5,1,61,3,0,3,0,9,...,0.000000e+00,1.196468e-07,0.000000e+00,6.067383e-07,0.000000e+00,2.123537e-07,8.895046e-06,0.000010,1.698422e-06,False
8451,alchimius,0,4,1,2,0,2,0,0,1,...,6.687313e-08,0.000000e+00,0.000000e+00,6.741536e-08,0.000000e+00,0.000000e+00,4.851843e-07,0.000010,8.144477e-07,False


In [154]:
target = "equus"
base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
resp_json = requests.get(base_url.format(target)).json()
resp_json

[{'id': 63,
  'intern_type': 'noun',
  'short_name': 'equus',
  'full_name': 'equus, equi [m.] O',
  'type': {'name': 'noun', 'label': 'Noun'},
  'translations_unstructured': {'en': 'horse', 'de': 'Pferd'},
  'url': 'https://www.latin-is-simple.com/en/vocabulary/noun/63/'}]

In [155]:
resp_json[0]["translations_unstructured"]["en"]

'horse'

In [156]:
def simple_translation(target):
    try:
        base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
        resp_json = requests.get(base_url.format(target)).json()
        transl = resp_json[0]["translations_unstructured"]["en"]
        if transl == "still in translation":
            transl = ""
        return transl
    except:
        return ""

In [157]:
import concurrent.futures

def get_translations_parallel(df, col_name="word", n_workers=30):
    words = df[col_name].tolist()
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_workers) as executor:
        results = list(executor.map(simple_translation, words))
    return results

In [158]:
len(filtered_vocab_df)

11042

In [159]:
%%time
filtered_vocab_df["transl"] = get_translations_parallel(filtered_vocab_df)

CPU times: user 52.8 s, sys: 4.59 s, total: 57.4 s
Wall time: 2min 20s


In [160]:
filtered_vocab_df.sample(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
8417,uisorius,92,122,326,53,395,187,44,77,51,...,0.000002,0.000004,3.438184e-06,0.000019,0.000003,0.000003,0.000000e+00,0.000006,False,
9827,tractus,1236,1859,1309,2290,2563,1582,1733,613,1346,...,0.000069,0.000031,9.074108e-05,0.000047,0.000138,0.000057,5.092681e-05,0.000070,True,"draw, drag, derive, assume, acquire, influence..."
1502,uiduus,53,131,105,91,145,135,128,53,59,...,0.000005,0.000003,3.977506e-06,0.000004,0.000004,0.000005,5.722114e-07,0.000004,False,
4443,candidior,194,468,113,125,460,55,665,29,107,...,0.000027,0.000001,7.213444e-06,0.000002,0.000005,0.000002,2.861057e-06,0.000008,True,"shining, bright, white, beautiful"
10016,chorda,1046,790,640,1899,1013,2098,362,2311,362,...,0.000014,0.000117,2.440436e-05,0.000089,0.000031,0.000008,2.574951e-06,0.000046,False,"late-born/produced out of/late in season, seco..."
1221,imperfectio,19,210,175,183,187,280,133,161,113,...,0.000005,0.000008,7.617936e-06,0.000011,0.000008,0.000006,3.147162e-05,0.000009,True,imperfection
10214,elegans,259,548,588,702,1118,518,1124,421,540,...,0.000045,0.000021,3.640430e-05,0.000026,0.000048,0.000050,9.727593e-06,0.000029,True,"elegant, fine, handsome, tasteful, fastidious,..."
8429,assuetudo,8,46,47,43,94,51,50,39,9,...,0.000002,0.000002,6.067383e-07,0.000003,0.000003,0.000002,0.000000e+00,0.000002,False,"custom, habit, repeated practice/experience/as..."
10822,ossa,140,367,179,98,570,76,174,20,87,...,0.000007,0.000001,5.865137e-06,0.000003,0.000005,0.000006,8.010959e-06,0.000007,True,"bone, (implement, gnawed, dead), kernel (nut),..."
3995,exercitus,506,1646,525,1212,665,873,1134,512,818,...,0.000045,0.000026,5.514577e-05,0.000037,0.000134,0.000023,1.430528e-05,0.000042,True,"train, discipline, drill"


In [161]:
subcorpus_counts_df.head(5)

,subcorpus,sents_n,tokens_n,vectors_tokens,vectors_coverage
0,1501-1550,1018967,12779473,4564709,0.357191
1,1551-1600,2679563,35876603,12122038,0.337881
2,1601-1650,1423173,17932748,6182120,0.344739
3,1651-1700,2019943,27211868,9048914,0.332536
4,Medicine,3210357,36896246,12791137,0.346679


In [162]:
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921,True,"say, call, tell"
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680,True,"book, volume, inner bark of a tree, book, book"
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563,True,"be able, can"
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367,True,"do, make, handle"
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310,True,"have, hold, possess, consider, think"
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397,0.002279,True,part
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,...,0.001686,0.001846,0.002739,0.002701,0.002085,0.002239,0.002556,0.002129,True,suddenly
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,...,0.001883,0.001437,0.001897,0.002244,0.001466,0.001554,0.001730,0.001717,True,
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,...,0.001626,0.001578,0.001717,0.001703,0.002230,0.001240,0.001090,0.001632,True,"place, location"
56,aqua,11747,41231,20765,34038,42778,18714,30884,14948,28253,...,0.001232,0.000754,0.001905,0.001437,0.001160,0.001966,0.005758,0.001575,True,"get/fetch/bring water, be watered"


In [169]:
filtered_vocab_df = filtered_vocab_df[filtered_vocab_df["word"].apply(lambda x: len(x) > 2)]

In [192]:
filtered_vocab_df.to_json("../data/filtered_vocab_df.json")
filtered_vocab_df.to_json("/srv/webserver/apps/iweems_app/data/filtered_vocab_df.json")
filtered_vocab_df.to_json("~/notebooks/iweems_huggingface/data/filtered_vocab_df.json")

In [193]:
!cp ../data/filtered_vocab_df.json /srv/data/tome/noscemus/filtered_vocab_df.json

In [194]:
filtered_vocab_df.to_csv("../data/filtered_vocab_df.csv", index=False)
